In [1]:
from torch import nn

In [2]:
rnn=nn.RNN(100,10)  # 每个词向量特征维度，隐藏层维度
rnn._parameters.keys()
# l0表示现在只有一层，rnn可以有多层

odict_keys(['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0'])

In [3]:
rnn.weight_ih_l0.shape

torch.Size([10, 100])

In [4]:
rnn.weight_hh_l0.shape

torch.Size([10, 10])

In [5]:
rnn.bias_ih_l0.shape

torch.Size([10])

In [6]:
rnn.bias_hh_l0.shape

torch.Size([10])

# 整理以下
输入input形状应该是time(len)*batch*feature，其中time，batch不受网络形状约束，假设（5，2，100），time其实不作为计算输入，而是循环次数，实际计算维度（2，100）
W_ih形状hidden*feature，（10，100）
W_hh形状hidden*hidden，（10，10）
h-存放历史信息，在0时刻应该初始化，后来的形状是batch*hidden，（2，10）
首先更新h，h=X@W_ih+h@W_hh=(2,100)(100,10)+(2,10)(10,10)=(2,10)

In [7]:
rnn1=nn.RNN(input_size=100,hidden_size=20,num_layers=1)
import torch
x=torch.rand(10,3,100)
out,h=rnn1(x,torch.zeros(1,3,20))  # 每次执行时，都要初始化隐藏层。输出结果h最后时刻隐藏层信息，out每一时刻隐藏层信息，h.shape-(1,3,20),out.shape-(10,3,20)。一层就设置1，n层设置n
out.shape,h.shape

(torch.Size([10, 3, 20]), torch.Size([1, 3, 20]))

In [8]:
# 实验两层rnn
rnn2=nn.RNN(input_size=100,hidden_size=10,num_layers=2)

In [9]:
rnn2._parameters.keys()

odict_keys(['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0', 'weight_ih_l1', 'weight_hh_l1', 'bias_ih_l1', 'bias_hh_l1'])

In [10]:
rnn2.weight_ih_l0.shape,rnn2.weight_hh_l0.shape,rnn2.weight_ih_l1.shape,rnn2.weight_hh_l1.shape
# 第二层的输入是第一层的h

(torch.Size([10, 100]),
 torch.Size([10, 10]),
 torch.Size([10, 10]),
 torch.Size([10, 10]))

In [11]:
rnn4=nn.RNN(input_size=100,hidden_size=20,num_layers=4)
print(rnn4)

RNN(100, 20, num_layers=4)


In [12]:
x=torch.rand(10,3,100)
out,h=rnn4(x)
out.shape,h.shape  # out每个时间维度上，h每层的最后时间维度上

(torch.Size([10, 3, 20]), torch.Size([4, 3, 20]))

In [13]:
# RNNCell使用
# 同RNN来说，对于每个时间步的计算需要手写
cell1=nn.RNNCell(100,20)  # feature,hidden
h1=torch.zeros(3,20) # batch,hidden  ->(1,3,20)说明只有一层
for x_t in x:
    h1=cell1(x_t,h1)

h1.shape

torch.Size([3, 20])

In [14]:
x.shape

torch.Size([10, 3, 100])

In [15]:
# 两层RNNCell
cell1=nn.RNNCell(100,30)
ce112=nn.RNNCell(30,20)
h1=torch.zeros(3,30)  # 因为是一层，默认不用写成（1，3，20）
h2=torch.zeros(3,20)
for x_t in x:
    h1=cell1(x_t,h1)  # 3*30
    h2=ce112(h1,h2)  # 3*20
h2.shape

torch.Size([3, 20])

In [16]:
# RNN预测正弦曲线
import numpy as np
start=np.random.randint(3,size=1)[0] # 生成大小为1，值介于0，3之间的数组，并取第一个数
time_steps=np.linspace(start,start+10,50)  # 在start和start+10之间均匀生成50个点，作为训练的样本

In [17]:
# 根据上文的训练样本，得到y
data=np.sin(time_steps)
print(data.shape)
data=data.reshape(50,1)
data.shape

(50,)


(50, 1)

In [18]:
# 制作训练集
# 训练集50个样本，下标从0-48，标签从1-49
x=torch.tensor(data[:-1]).float().view(1,49,1)
y=torch.tensor(data[1:]).float().view(1,49,1)
x.shape,y.shape  # batch,time,feature(因为是预测具体值，所以直接表示即可)

(torch.Size([1, 49, 1]), torch.Size([1, 49, 1]))

In [19]:
class net(nn.Module):
    def __init__(self,):
        super(net,self).__init__()
        self.rnn=nn.RNN(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=1,
                        batch_first=True
                        )
        self.linear=nn.Linear(hidden_size,output_size)

    def forward(self,x,hidden_prev):
        out,hidden_prev=self.rnn(x,hidden_prev)
        out=out.view(-1,hidden_size)
        out=self.linear(out)
        out=out.unsqueeze(dim=0)
        return out,hidden_prev